***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [6. Deconvolution in Imaging](6_0_introduction.ipynb)  
    * Previous: [6.1 Sky Models](6_1_sky_models.ipynb)  
    * Next: [6.3 Residuals and Image Quality](6_3_residuals_and_iqa.ipynb)

***

Import standard modules:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Import section specific modules:

In [ ]:
from IPython.display import Image
from IPython.core.display import clear_output, display

from scipy.signal import convolve2d

from copy import copy
import time

import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 6.0)
import matplotlib.cm as cm

***

## 6.2 Point Source Assumption (CLEAN)<a id='deconv:sec:clean'></a>

**TODO**

* naive deconvolution approach: broad threshold
* setup the concept of clean: everything is a point source in the image domain, we know what the PSF looks like (approximately), go through the image and find the peak, subtract off the PSF centred on that pixel (just a precentage of the flux), and repeat until happy (ill-defined)
* Laura's 2D deconvolution example
* imager-based 2D set through example
* describe image-domain CLEAN (Hogbom), limits of possible clean region, advantages in speed
* extend to gridded vis-domain (Clark), limits in errors due to gridding, advantage in speed and no region issue
* extend to ungridded vis-domain (Cotton-Schwab), requires a de-gridder, computationally more expensive, but has the best results
* show the effect in the visibility domain, converts the tracks into a Gaussian distribution, introduce the restoring beam
* limits of clean: complex sources, large-FOV varying PSF ; current work in compressed sensing and bayesian model selection (don't actually need an image to recover the science)
* extra issues: multi-frequency, multi-scale synthesis, list of current imagers
* lead into residuals and IQA: what makes a good image?

***

In [ ]:
from IPython.core.display import HTML 
styles = open("../style/course.css", "r").read() # read course.css file from root dir
HTML(styles) # apply style to page

## 6.2.1 Sampling, the PSF and the dirty image<a id='deconv:sec:sampling'></a>

In Chapter 5.2 we discussed the effects of sampling on our imaging. When we observe with a radio interferometer we collect visibility data at a range of $uv$ coordinates, which we then Fourier Transform into an image. The $uv$ coordniates of the visibility data don't fully cover the $uv$-plane. We saw examples of this in plots of KAT-7 $uv$ converage. For example:

In [ ]:
#Image(filename='figures/uvcoverage/KAT-7_6h60s_dec-30_10MHz_10chans.png')

Conceptually we can think of this partial $uv$ coverage as the product of the complete visibilities with full $uv$ coverage filling the UV-plane $V(u,v)$, with a sampling function $S(u,v)$ that is unity where we have $uv$ samples and zero where we don't:

$$ S(u,v) \times V(u,v) $$

Our image on the sky is the Fourier Transform of the conceptual complete visibility:

$$ \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} V(u,v) \; e^{-2\pi i(ul+vm)} \;\;\;\;\; du \, dv =  I(l,m)$$

But what we get from the interferometer isn't $V(u,v)$, it is the incomplete $uv$ coverage product $S(u,v) \times V(u,v)$:

$$ \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} S(u,v) \times V(u,v) \; e^{-2\pi i(ul+vm)} \;\;\;\;\; du \, dv \;=\; ?? $$

How do we deal with this? Remember the [Convolution Theorem &#10142;](http://mathworld.wolfram.com/ConvolutionTheorem.html#destination), which tells us that multiplication in one Fourier domain is equivalent to convolution in the other Fourier domain:

$$ h \times g \;\; \rightleftharpoons \;\; H \; . \; G $$

In our incomplete $uv$-coverage situation, we are multiplying for conceptual complete visibilities $V(u,v)$ by the sampling function $S(u,v)$, which equates to a convolution in the image domain:

$$ S(u,v) \times V(u,v) \;\; \rightleftharpoons \;\; B(l,m) \; * \; I(l,m) $$

where $I(l,m)$ is the true sky image, the Fourier Transform of the complete visibility $V(u,v)$, and $B(l,m)$ is the Fourier Transform of the sampling function $S(u,v)$.

We term the product $B(l,m) \; * \; I(l,m)$ the "Dirty Image" $I^D(l,m)$ and we term $B(l,m)$ the "Dirty Beam" or point spread function (PSF).

<p class=conclusion>
  <font size=4> **Summary**</font>
  <br>
  <br>
&bull; 
The Fourier Transform of our sampled visibility $V(u,v)$ data gives us a dirty image $I^D(l,m)$
<br><br>
&bull; 
The dirty image $I^D(l,m)$ is the convolution of the true image $I(l,m)$ with the PSF
<br><br>
&bull;
The PSF (or dirty beam $B(l,m)$) is the Fourier Transform of the sampling function $S(u,v)$
</p>

So how do we extract the true sky image $I(l,m)$ from the dirty image $I^D(l,m)$? We have to **deconvolve** the PSF and the dirty image.

***

## 6.2.2 Point Source Assumption<a id='deconv:sec:point_source'></a>

One convenient feature of radio astronomy images is that they consist *for the most part* of point sources on an empty sky. This is just an approximation of course, but it is an approximation that makes decomvolution much easier, because we can then consider our PSF as a point source response (or impulse response) - every point source on the true image is replaced by a PSF in the dirty image. 

Let's set up a little example of this.

First, set up a toy sky, with three point sources and some noise:

In [ ]:
imsize = 50

# create noise background
noise_rms = 0.1
I = noise_rms*(np.random.random([imsize,imsize])-0.5)

# add three point sources
I[20,20] += 1
I[32,15] += 1.45
I[30,34] += 1.12

plt.imshow(I, cmap=cm.jet)
plt.colorbar()
plt.title('I(l,m)');

Now set up a fake PSF. We will just make up a star shape to be the PSF:

In [ ]:
PSFsize = 13
PSF = np.zeros([PSFsize,PSFsize])
PSFmid = (PSFsize - 1)/2
PSF[:,PSFmid] = 0.5
PSF[PSFmid,:] = 0.5
d1, d2 = np.diag_indices_from(PSF)
PSF[d1,d2] = 0.5
PSF[d1,d2[::-1]] = 0.5
PSF[PSFmid-2:PSFmid+3,PSFmid-2:PSFmid+3] = 0
PSF[PSFmid-1:PSFmid+2,PSFmid-1:PSFmid+2] = 0.75
PSF[PSFmid,PSFmid] = 1.0

plt.imshow(PSF, cmap = cm.jet)
plt.colorbar()
plt.title('PSF(l,m)');

Now we convolve our true sky image $I(l,m)$ with the PSF, to give the dirty image $I^D(l,m)$

In [ ]:
I_dirty = convolve2d(I,PSF,mode='same')

Lets plot all three together to see what we have just done:

In [ ]:
fig, axes = plt.subplots(figsize=(16,16))

plt.subplot(131)
plt.imshow(I, cmap=cm.jet)
plt.title('I(l,m)')

plt.subplot(132)
plt.imshow(PSF, cmap=cm.jet)
plt.xlim(-15,25)
plt.ylim(-15,25)
plt.title('PSF(l,m)')

plt.subplot(133)
plt.imshow(I_dirty, cmap=cm.jet)
plt.title('Dirty image I^D(l,m)');

***

## 6.2.3 CLEAN <a id='deconv:sec:clean'></a>

CLEAN is an algorithm to deconvolve the dirty image and PSF. The CLEAN algorithm was originally devised by Högbom (1974) and more sophisticated versions were developed later. We will describe the Högbom CLEAN here and discuss more recent CLEAN algorithms in the next section.

In the Högbom CLEAN, the deconvolution is performed by iteratively finding peaks in the image and removing scaled versions of the PSF at the peak positions. It is an image-based CLEAN, meaning that the deconvolution is done entirely in the image $lm$ plane (some later CLEAN algorithms also work in part in the visibility $uv$ plane).

A summary of the algorithm can be found in The White Book (pg 154).

*** Högbom's Algorithm (Image-domain CLEAN): ***

1. Make a copy the dirty image $I^D(l,m)$ called the residual image $I^R(l,m)$
2. Find the strength and position of the peak in the residual image $I^R(l,m)$
3. Subtract from the residual image $I^R(l,m)$, at the position of the peak, the dirty beam $B(l,m)$ multiplied by the peak strength and a gain factor $g$.
4. Record the position and magnitude of the point source subtracted in a model
5. Go to (2.) unless any remaining peak is below some user-specified threshold or the number of iterations reached some user-specified limit.
6. Convolve the accumulated point source model with a restoring beam, termed the CLEAN beam (usually a Gaussian fitted to the main lobe of the dirty beam)
7. Add the remainder of the residual image $I^R(l,m)$ to the CLEAN image formed in (6.) to form the final restored image

Input: Dirty image, dirty beam (PSF)

Parameters: gain, iteration limit, flux threshold

Output: Model, residual image, restored image

Let us use the algorithm steps above to deconvolve our dirty image from the previous section:

In [ ]:
# ------------------------------------------------
# Step 1: copy the dirty image to a residual image
# ------------------------------------------------
I_residual = copy(I_dirty)

# set up the input parameters 
#   (you can change these later to see how they impact the algorithm)
gain = 0.2
niter = 100
threshold = 5.*noise_rms

plotmax = np.max(I)
plotmin = np.min(I)
model = []

In [ ]:
# plot dirty image to compare to the residual image as we run the algorithm
f, ax = plt.subplots(1,2,figsize=[16,6])
ax[0].set_title('I(l,m)')
ax[0].imshow(I_dirty, cmap=cm.jet, vmax=plotmax, vmin=plotmin);

for i in range(niter):  
    print 'Iteration {0}:'.format(i,)
    
    # ------------------------------------------------
    # Step 2. Find the strength and position of the peak in the residual image
    # ------------------------------------------------
    f_max = np.max(I_residual)
    p_max = np.where(I_residual==f_max)
    
    # ------------------------------------------------
    # Step 3. Subtract gain*PSF centred on $p_{max}$ from the residual image
    # ------------------------------------------------    
    p_x, p_y = p_max
    I_residual[p_x-PSFmid:p_x+PSFmid+1,p_y-PSFmid:p_y+PSFmid+1] -= gain*PSF
    print 'Peak: {0}    Position: {1},{2}'.format(f_max,p_x[0],p_y[0])
    
    # ------------------------------------------------
    # Step 4. Record the peak position and the magnitude subtracted in the model
    # ------------------------------------------------        
    model.append([p_x[0], p_y[0], gain*f_max])
    
    # ------------------------------------------------
    # Step 5. Repeat from (2.), unless residual image < threshold
    # ------------------------------------------------      
    if np.max(I_residual) < threshold: 
        print 'Residual map peak is less than threshold {0}'.format(threshold,)
        break

    # plot the new residial next to the original image 
    ax[1].imshow(I_residual, cmap=cm.jet, vmax=plotmax, vmin=plotmin)
    ax[1].set_title('I_residual(l,m)')    
    # show the plot, then get ready for the next plot
    plt.draw()
    clear_output(wait=True)
    time.sleep(0.2)
    display(f)
    ax[1].cla()
    plt.close()  

Lets plot the original image and the final residual image scaled to see the residuals:

In [ ]:
plotmax = np.max(I_residual)
plotmin = np.min(I_residual)

fig, axes = plt.subplots(figsize=(16,6))

plt.subplot(121)
plt.imshow(I_dirty, cmap=cm.jet, vmax=plotmax, vmin=plotmin)
plt.colorbar()

plt.subplot(122)
plt.colorbar()
plt.imshow(I_residual, cmap=cm.jet, vmax=plotmax, vmin=plotmin);

In [ ]:
# now sum the accumulated point source model ("clean components") into a model image
print 'Clean components:'
print 'x  y  flux'

I_model = np.zeros([imsize,imsize])
for x, y, f in model:
    print x, y, f
    I_model[x,y] += f

And plot the deconvolved model image next to the original true image

In [ ]:
plotmax = np.max(I)
plotmin = np.min(I)
fig, axes = plt.subplots(figsize=(16,6))

plt.subplot(121)
plt.imshow(I, cmap=cm.jet, vmax=plotmax, vmin=plotmin)
plt.colorbar()

plt.subplot(122)
plt.imshow(I_model, cmap=cm.jet, vmax=plotmax, vmin=plotmin)
plt.colorbar();

Steps 6 and 7 in the CLEAN algorithm is convolving the model by a restoring beam, and adding back the residual image. The restoring beam is usualy just a Gaussian fit to the main beam. 

In our toy example here it is hardly worth doing a proper Gaussian fit, but let's do it anyway:

In [ ]:
# first get just the main lobe of the star shaped PSF
main_lobe = np.zeros([PSFsize,PSFsize])
main_lobe[PSFmid-1:PSFmid+2,PSFmid-1:PSFmid+2] = 0.75
main_lobe[PSFmid,PSFmid] = 1.0

fig, axes = plt.subplots(figsize=(16,6))
plt.subplot(121)
plt.imshow(PSF, cmap=cm.jet)
plt.colorbar()
plt.title('PSF(l,m)');

plt.subplot(122)
plt.imshow(main_lobe, cmap=cm.jet)
plt.colorbar()
plt.title('main lobe(l,m)');

In [ ]:
# now fit a symmetric 2D gaussian to the main lobe
import scipy.optimize as opt

def gaussian2dsymmetric((x,y),A,x0,y0,sigma):
    gauss2d = A*np.exp(-((x-x0)**2.0 + (y-y0)**2.0)/(2.*sigma**2.0))
    return gauss2d.ravel()

x,y = np.meshgrid(range(PSFsize),range(PSFsize))
popt, pcov = opt.curve_fit(gaussian2dsymmetric,(x, y),main_lobe.ravel(), p0=[1.0,6.5,6.5,2.])
A, x0, y0, sigma = popt
print "Fit results:"
print "A: {0},  x0: {1}  y0: {2}  sigma: {3}".format(A,x0,y0,sigma)

# use fitted values to create CLEAN beam (or restoring beam)
#   normalise by dividing through by A
clean_beam = gaussian2dsymmetric((x,y),A,x0,y0,sigma).reshape(PSFsize,PSFsize)/A

In [ ]:
# plot the CLEAN beam
plt.imshow(clean_beam, cmap=cm.jet)
plt.colorbar()
plt.title('CLEAN beam(l,m)');

And now the final steps in the CLEAN are to convolve our model by the CLEAN beam and add back the residuals:

In [ ]:
# ------------------------------------------------
# Step 6: convolve the model with the CLEAN beam
# ------------------------------------------------
I_restored = convolve2d(I_model,clean_beam,mode='same') 

# ------------------------------------------------
# Step 7: add the residuals back to the restored image
# ------------------------------------------------
I_restored = I_restored + I_residual

Now let us plot the results of the CLEAN:

In [ ]:
plotmax = np.max(I_dirty)
plotmin = np.min(I_dirty)

fig, axes = plt.subplots(figsize=(16,12))

plt.subplot(221)
plt.imshow(I, cmap=cm.jet, vmax=plotmax, vmin=plotmin)
plt.title('I(l,m)')
plt.colorbar()

plt.subplot(222)
plt.colorbar()
plt.title('I_dirty(l,m)')
plt.imshow(I_dirty, cmap=cm.jet, vmax=plotmax, vmin=plotmin);

plt.subplot(223)
plt.title('I_model(l,m)')
plt.imshow(I_model, cmap=cm.jet, vmax=plotmax, vmin=plotmin)
plt.colorbar()

plt.subplot(224)
plt.colorbar()
plt.title('I_restored(l,m)');
plt.imshow(I_restored, cmap=cm.jet, vmax=plotmax, vmin=plotmin);

Notice how the CLEAN is imperfect!

Even in a simple field of distant point sources, with no noise, PSF side lobes can overlap and add up and CLEAN can wrongly identify them as sources. And as soon as we add noise the situation degrades further - when we start CLEANing to close to the noise level the CLEAN can start picking up noise as sources. Also individual point sources can be subtracted in multiple components in adjacent pixels.

But having said that - CLEAN usually performs adequately. As long as we are careful to look at the results of our CLEAN, and are aware of image artifacts and noise levels!

There are also lots of parameters to fiddle with, even in our simple clean. 

Try changing the gain and cutoff threshold and re-run this notebook, to see how they effect the CLEAN. Also try changing the noise level of the original map by changing noise_rms - you will see how the CLEAN performs better when the signal to noise ratio is higher.

***

Next: [6.3 Residuals and Image Quality](6_3_residuals_and_iqa.ipynb)